In [1]:
import pandas as pd

In [7]:
df = pd.read_excel("default of credit card clients.xls")
df.head()

ImportError: Pandas requires version '2.0.1' or newer of 'xlrd' (version '1.2.0' currently installed).